In [1]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType, AccumulateExitMode, ConflictMode, CallSeqType

import numpy as np
import pandas as pd
from numba import njit
from datetime import datetime, timedelta

seed = 42

price = pd.Series([1., 2., 3., 4., 5.], index=pd.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5)
]))
price_wide = price.vbt.tile(3, keys=['a', 'b', 'c'])
print(price.shape, price_wide.shape)

big_price = pd.DataFrame(np.random.uniform(size=(1000,)))
big_price.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]
big_price_wide = big_price.vbt.tile(1000)
print(big_price.shape, big_price_wide.shape)

# Disable caching for performance testing
# NOTE: Expect waterfall of executions, since some attributes depend on other attributes 
# that have to be calculated again and again
vbt.defaults.caching = False

(5,) (5, 3)
(1000, 1) (1000, 1000)


## from_signals

In [2]:
entries = pd.Series([True, True, True, False, False], index=price.index)
entries_wide = entries.vbt.tile(3, keys=['a', 'b', 'c'])
print(entries.shape, entries_wide.shape)

exits = pd.Series([False, False, True, True, True], index=price.index)
exits_wide = exits.vbt.tile(3, keys=['a', 'b', 'c'])
print(exits.shape, exits_wide.shape)

big_entries = pd.DataFrame.vbt.signals.empty((1000,), index=big_price.index)
big_entries.iloc[0::2] = True
big_entries_wide = big_entries.vbt.tile(1000)
print(big_entries.shape, big_entries_wide.shape)

big_exits = pd.DataFrame.vbt.signals.empty((1000,), index=big_price.index)
big_exits.iloc[1::2] = True
big_exits_wide = big_exits.vbt.tile(1000)
print(big_exits.shape, big_exits_wide.shape)

(5,) (5, 3)
(5,) (5, 3)
(1000,) (1000, 1000)
(1000,) (1000, 1000)


In [4]:
%timeit vbt.Portfolio.from_signals(\
    big_price, big_entries, big_exits,\
    size=1., init_cash=np.inf, accumulate=True)

2.17 ms ± 86.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True)

78.1 ms ± 9.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.repeat(np.arange(500), 2))

75.6 ms ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0))

74.5 ms ± 6.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0), cash_sharing=True)

79.7 ms ± 9.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits)

print(portfolio.orders.records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1


In [10]:
portfolio = vbt.Portfolio.from_signals(price_wide, entries, exits)

print(portfolio.orders.records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    1    0  100.0    1.0   0.0     0
3    1    3  100.0    4.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    3  100.0    4.0   0.0     1


In [11]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]))

print(portfolio.orders.records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    1    0  100.0    1.0   0.0     0
3    1    3  100.0    4.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    3  100.0    4.0   0.0     1


In [12]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders.records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  0  1  0
2020-01-05  0  1  0


In [13]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Reversed)

print(portfolio.orders.records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    1    0  100.0    1.0   0.0     0
1    1    3  100.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [14]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Random)

print(portfolio.orders.records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  1  0  0
2020-01-05  0  1  0


In [15]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1., fees=0.01)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    3   1.0    4.0  0.04     1


In [16]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1., fixed_fees=1.)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   1.0     0
1    0    3   1.0    4.0   1.0     1


In [17]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1., slippage=0.01)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0   1.01   0.0     0
1    0    3   1.0   3.96   0.0     1


In [18]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., 
    entry_price=price*1.01, exit_price=price*0.99)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0   1.01   0.0     0
1    0    3   1.0   3.96   0.0     1


In [19]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., 
    size_type=SizeType.Shares, accumulate=False)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1


In [20]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., 
    size_type=SizeType.Cash, accumulate=False)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1


In [21]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Shares, 
    accumulate=True, accumulate_exit_mode=AccumulateExitMode.Close)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     0
2    0    3   2.0    4.0   0.0     1


In [22]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Cash, 
    accumulate=True, accumulate_exit_mode=AccumulateExitMode.Close)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   0.5    2.0   0.0     0
2    0    3   1.5    4.0   0.0     1


In [23]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Shares, 
    accumulate=True, accumulate_exit_mode=AccumulateExitMode.Reduce)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     0
2    0    3   1.0    4.0   0.0     1
3    0    4   1.0    4.0   0.0     1


In [24]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Cash, 
    accumulate=True, accumulate_exit_mode=AccumulateExitMode.Reduce)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0  1.00    1.0   0.0     0
1    0    1  0.50    2.0   0.0     0
2    0    3  0.25    4.0   0.0     1
3    0    4  0.25    4.0   0.0     1


In [25]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Shares, 
    accumulate=True, conflict_mode=ConflictMode.Exit)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     0
2    0    2   2.0    3.0   0.0     1


In [26]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Cash, 
    accumulate=True, conflict_mode=ConflictMode.Exit)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   0.5    2.0   0.0     0
2    0    2   1.5    3.0   0.0     1


In [27]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Shares, 
    accumulate=True, conflict_mode=ConflictMode.ExitAndEntry)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     0
2    0    2   1.0    3.0   0.0     1
3    0    3   1.0    4.0   0.0     1


In [28]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., size_type=SizeType.Cash, 
    accumulate=True, conflict_mode=ConflictMode.ExitAndEntry)

print(portfolio.orders.records)

   col  idx      size  price  fees  side
0    0    0  1.000000    1.0   0.0     0
1    0    1  0.500000    2.0   0.0     0
2    0    2  1.166667    3.0   0.0     1
3    0    3  0.333333    4.0   0.0     1


## from_orders

In [49]:
order_size = pd.Series([np.inf, -np.inf, np.nan, np.inf, -np.inf], index=price.index)
order_size_wide = order_size.vbt.tile(3, keys=['a', 'b', 'c'])

big_order_size = pd.DataFrame.vbt.empty((1000,), 1, dtype=np.float_)
big_order_size.iloc[1::2] = -1
big_order_size_wide = big_order_size.vbt.tile(1000)

In [30]:
%timeit vbt.Portfolio.from_orders(big_price, big_order_size, init_cash=np.inf)

1.66 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit vbt.Portfolio.from_orders(big_price_wide, big_order_size_wide, init_cash=np.inf)

70.9 ms ± 7.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.repeat(np.arange(500), 2))

70.2 ms ± 9.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0))

66.5 ms ± 5.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True)

73.9 ms ± 8.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
portfolio = vbt.Portfolio.from_orders(price, order_size)

print(portfolio.orders.records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  100.0    2.0   0.0     1
2    0    3   50.0    4.0   0.0     0
3    0    4   50.0    4.0   0.0     1


In [51]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size)

print(portfolio.orders.records)

    col  idx   size  price  fees  side
0     0    0  100.0    1.0   0.0     0
1     0    1  100.0    2.0   0.0     1
2     0    3   50.0    4.0   0.0     0
3     0    4   50.0    4.0   0.0     1
4     1    0  100.0    1.0   0.0     0
5     1    1  100.0    2.0   0.0     1
6     1    3   50.0    4.0   0.0     0
7     1    4   50.0    4.0   0.0     1
8     2    0  100.0    1.0   0.0     0
9     2    1  100.0    2.0   0.0     1
10    2    3   50.0    4.0   0.0     0
11    2    4   50.0    4.0   0.0     1


In [52]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]))

print(portfolio.orders.records)

    col  idx   size  price  fees  side
0     0    0  100.0    1.0   0.0     0
1     0    1  100.0    2.0   0.0     1
2     0    3   50.0    4.0   0.0     0
3     0    4   50.0    4.0   0.0     1
4     1    0  100.0    1.0   0.0     0
5     1    1  100.0    2.0   0.0     1
6     1    3   50.0    4.0   0.0     0
7     1    4   50.0    4.0   0.0     1
8     2    0  100.0    1.0   0.0     0
9     2    1  100.0    2.0   0.0     1
10    2    3   50.0    4.0   0.0     0
11    2    4   50.0    4.0   0.0     1


In [53]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders.records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  100.0    2.0   0.0     1
2    0    3   50.0    4.0   0.0     0
3    0    4   50.0    4.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    1  100.0    2.0   0.0     1
6    2    3   50.0    4.0   0.0     0
7    2    4   50.0    4.0   0.0     1
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  0  1  0
2020-01-05  0  1  0


In [54]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Reversed)

print(portfolio.orders.records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    1    0  100.0    1.0   0.0     0
1    1    1  100.0    2.0   0.0     1
2    1    3   50.0    4.0   0.0     0
3    1    4   50.0    4.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    1  100.0    2.0   0.0     1
6    2    3   50.0    4.0   0.0     0
7    2    4   50.0    4.0   0.0     1
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [56]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Random)

print(portfolio.orders.records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  100.0    2.0   0.0     1
2    0    3   50.0    4.0   0.0     0
3    0    4   50.0    4.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    1  100.0    2.0   0.0     1
6    2    3   50.0    4.0   0.0     0
7    2    4   50.0    4.0   0.0     1
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  1  0  0
2020-01-04  0  1  0
2020-01-05  1  0  0


In [61]:
order_size_one = pd.Series([1, -1, np.nan, 1, -1], index=price.index)

In [62]:
portfolio = vbt.Portfolio.from_orders(price, order_size_one, fees=0.01)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    1   1.0    2.0  0.02     1
2    0    3   1.0    4.0  0.04     0
3    0    4   1.0    4.0  0.04     1


In [63]:
portfolio = vbt.Portfolio.from_orders(price, order_size_one, fixed_fees=1.)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   1.0     0
1    0    1   1.0    2.0   1.0     1
2    0    3   1.0    4.0   1.0     0
3    0    4   1.0    4.0   1.0     1


In [64]:
portfolio = vbt.Portfolio.from_orders(price, order_size_one, slippage=0.01)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0   1.01   0.0     0
1    0    1   1.0   1.98   0.0     1
2    0    3   1.0   4.04   0.0     0
3    0    4   1.0   3.96   0.0     1


In [65]:
portfolio = vbt.Portfolio.from_orders(price, order_size_one, size_type=SizeType.Shares)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     1
2    0    3   1.0    4.0   0.0     0
3    0    4   1.0    4.0   0.0     1


In [66]:
portfolio = vbt.Portfolio.from_orders(price, order_size_one, size_type=SizeType.Cash)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0  1.00    1.0   0.0     0
1    0    1  0.50    2.0   0.0     1
2    0    3  0.25    4.0   0.0     0
3    0    4  0.25    4.0   0.0     1


In [97]:
portfolio = vbt.Portfolio.from_orders(price, 50., size_type=SizeType.TargetShares)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0  50.0    1.0   0.0     0


In [96]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, 50., size_type=SizeType.TargetShares, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0  50.0    1.0   0.0     0
1    1    0  50.0    1.0   0.0     0
2    2    0  50.0    1.0   0.0     0


In [95]:
portfolio = vbt.Portfolio.from_orders(price, 50., size_type=SizeType.TargetCash)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0  50.0    1.0   0.0     0


In [94]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, 50., size_type=SizeType.TargetCash, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders.records)

   col  idx  size  price  fees  side
0    0    0  50.0    1.0   0.0     0
1    2    0  50.0    1.0   0.0     0


In [101]:
portfolio = vbt.Portfolio.from_orders(price, 100., size_type=SizeType.TargetValue)

print(portfolio.orders.records)

   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1   50.000000    2.0   0.0     1
2    0    2   16.666667    3.0   0.0     1
3    0    3    8.333333    4.0   0.0     1


In [100]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, 100., size_type=SizeType.TargetValue, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders.records)

    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1   50.000000    2.0   0.0     1
2     0    2   16.666667    3.0   0.0     1
3     0    3    8.333333    4.0   0.0     1
4     1    1   50.000000    2.0   0.0     0
5     1    2   16.666667    3.0   0.0     1
6     1    3    8.333333    4.0   0.0     1
7     2    0  100.000000    1.0   0.0     0
8     2    1   50.000000    2.0   0.0     1
9     2    2   16.666667    3.0   0.0     1
10    2    3    8.333333    4.0   0.0     1


In [102]:
portfolio = vbt.Portfolio.from_orders(price, 0.5, size_type=SizeType.TargetPercent)

print(portfolio.orders.records)

   col  idx      size  price  fees  side
0    0    0  50.00000    1.0   0.0     0
1    0    1  12.50000    2.0   0.0     1
2    0    2   6.25000    3.0   0.0     1
3    0    3   3.90625    4.0   0.0     1


In [103]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, 0.5, size_type=SizeType.TargetPercent, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders.records)
# warning! while each asset sees all available cash, it only sees its own shares & price

    col  idx       size  price  fees  side
0     0    0  50.000000    1.0   0.0     0
1     0    1  18.750000    2.0   0.0     1
2     0    2   6.250000    3.0   0.0     1
3     0    3   2.539062    4.0   0.0     1
4     0    4   1.025391    4.0   0.0     0
5     1    0  25.000000    1.0   0.0     0
6     1    1   3.125000    2.0   0.0     0
7     1    2   1.562500    3.0   0.0     1
8     1    3   2.050781    4.0   0.0     1
9     1    4   0.512695    4.0   0.0     1
10    2    0  50.000000    1.0   0.0     0
11    2    1  12.500000    2.0   0.0     1
12    2    2   6.250000    3.0   0.0     1
13    2    3   3.906250    4.0   0.0     1


## from_order_func

In [2]:
@njit
def order_one_func_nb(oc, price):
    if oc.i % 2 == 0:
        order_size = 1.
    else:
        order_size = -1.
    if price.ndim == 1:  # flexible indexing
        order_price = price[oc.i]
    else:
        order_price = price[oc.i, oc.col]
    return vbt.portfolio.nb.Order(order_size, SizeType.Shares, order_price, 0., 0., 0.)

@njit
def order_inf_func_nb(oc, price):
    if oc.i % 2 == 0:
        order_size = np.inf
    else:
        order_size = -np.inf
    if price.ndim == 1:  # flexible indexing
        order_price = price[oc.i]
    else:
        order_price = price[oc.i, oc.col]
    return vbt.portfolio.nb.Order(order_size, SizeType.Shares, order_price, 0., 0., 0.)

In [143]:
%timeit vbt.Portfolio.from_order_func(\
    big_price, order_one_func_nb, big_price, init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price, order_one_func_nb, big_price, init_cash=np.inf, row_wise=True)

The slowest run took 15.77 times longer than the fastest. This could mean that an intermediate result is being cached.
2.65 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
539 µs ± 116 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [144]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, init_cash=np.inf, row_wise=True)

105 ms ± 8.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
114 ms ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [145]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, \
    group_by=np.repeat(np.arange(500), 2), init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, \
    group_by=np.repeat(np.arange(500), 2), init_cash=np.inf, row_wise=True)

84.6 ms ± 7.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
95.6 ms ± 7.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [146]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0))
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), row_wise=True)

71.2 ms ± 5.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
73.8 ms ± 7.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [148]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_one_func_nb, big_price_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True, row_wise=True)

76.4 ms ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
83.8 ms ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def test_call_seq(call_seq, **kwargs):
    return vbt.Portfolio.from_order_func(
        price_wide, 
        order_one_func_nb, 
        price_wide, 
        group_by=np.array([0, 0, 0]),
        call_seq=call_seq, **kwargs
    ).call_seq

def big_test_call_seq(call_seq, **kwargs):
    return vbt.Portfolio.from_order_func(
        big_price_wide, 
        order_one_func_nb, 
        big_price_wide, 
        group_by=np.repeat(np.arange(500), 2),
        call_seq=call_seq, **kwargs
    ).call_seq

In [150]:
print(test_call_seq(CallSeqType.Default))
%timeit big_test_call_seq(CallSeqType.Default)

print(test_call_seq(CallSeqType.Reversed))
%timeit big_test_call_seq(CallSeqType.Reversed)

print(test_call_seq(CallSeqType.Random))
%timeit big_test_call_seq(CallSeqType.Random)

print(test_call_seq(42))
%timeit big_test_call_seq(42)

print(test_call_seq(test_call_seq(42)))
_call_seq = big_test_call_seq(42)
%timeit big_test_call_seq(_call_seq)

            a  b  c
2020-01-01  0  1  2
2020-01-02  0  1  2
2020-01-03  0  1  2
2020-01-04  0  1  2
2020-01-05  0  1  2
86.4 ms ± 6.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
87 ms ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  0  2  1
2020-01-02  2  1  0
2020-01-03  1  0  2
2020-01-04  0  2  1
2020-01-05  2  1  0
89.4 ms ± 6.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  0  1  2
2020-01-02  1  2  0
2020-01-03  0  1  2
2020-01-04  1  2  0
2020-01-05  0  1  2
89.2 ms ± 5.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  0  1  2
2020-01-02  1  2  0
2020-01-03  0  1  2
2020-01-04  1  2  0
2020-01-05  0  1  2
77 ms ± 5.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
@njit
def call_seq_func1_nb(grc, *args):
    return grc.call_seq[grc.i, grc.from_col:grc.to_col][::-1]

@njit
def call_seq_func2_nb(grc, *args):
    return np.arange(grc.to_col - grc.from_col - 1, -1, -1)

@njit
def row_prep_func_nb(grc, *args):
    call_seq_now = grc.call_seq[grc.i, grc.from_col:grc.to_col]
    grc.call_seq[grc.i, grc.from_col:grc.to_col] = call_seq_now[::-1]  # write to memory
    return ()

In [152]:
print(test_call_seq(CallSeqType.Default, call_seq_func_nb=call_seq_func1_nb, call_seq_args=()))  # fastest
%timeit big_test_call_seq(CallSeqType.Default, call_seq_func_nb=call_seq_func1_nb, call_seq_args=())

print(test_call_seq(CallSeqType.Default, call_seq_func_nb=call_seq_func2_nb, call_seq_args=()))
%timeit big_test_call_seq(CallSeqType.Default, call_seq_func_nb=call_seq_func2_nb, call_seq_args=())

print(test_call_seq(CallSeqType.Default, row_prep_func_nb=row_prep_func_nb, row_prep_args=()))
%timeit big_test_call_seq(CallSeqType.Default, row_prep_func_nb=row_prep_func_nb, row_prep_args=())

            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
78.9 ms ± 5.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
204 ms ± 4.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
201 ms ± 5.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [165]:
portfolio = vbt.Portfolio.from_order_func(price, order_inf_func_nb, price)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(price, order_inf_func_nb, price, row_wise=True)
print(portfolio.orders.records)

   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0


In [166]:
portfolio = vbt.Portfolio.from_order_func(price_wide, order_inf_func_nb, price_wide)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(price_wide, order_inf_func_nb, price_wide, row_wise=True)
print(portfolio.orders.records)

    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0   0.0     0
6     1    1  100.000000    2.0   0.0     1
7     1    2   66.666667    3.0   0.0     0
8     1    3   66.666667    4.0   0.0     1
9     1    4   53.333333    5.0   0.0     0
10    2    0  100.000000    1.0   0.0     0
11    2    1  100.000000    2.0   0.0     1
12    2    2   66.666667    3.0   0.0     0
13    2    3   66.666667    4.0   0.0     1
14    2    4   53.333333    5.0   0.0     0
    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0 

In [167]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide,
    group_by=np.array([0, 0, 1]))
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide,
    group_by=np.array([0, 0, 1]), row_wise=True)
print(portfolio.orders.records)

    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0   0.0     0
6     1    1  100.000000    2.0   0.0     1
7     1    2   66.666667    3.0   0.0     0
8     1    3   66.666667    4.0   0.0     1
9     1    4   53.333333    5.0   0.0     0
10    2    0  100.000000    1.0   0.0     0
11    2    1  100.000000    2.0   0.0     1
12    2    2   66.666667    3.0   0.0     0
13    2    3   66.666667    4.0   0.0     1
14    2    4   53.333333    5.0   0.0     0
    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0 

In [168]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, row_wise=True)
print(portfolio.orders.records)

   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
5    2    0  100.000000    1.0   0.0     0
6    2    1  100.000000    2.0   0.0     1
7    2    2   66.666667    3.0   0.0     0
8    2    3   66.666667    4.0   0.0     1
9    2    4   53.333333    5.0   0.0     0
   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
5    2    0  100.000000    1.0   0.0     0
6    2    1  100.000000    2.0   0.0     1
7    2    2   66.666667    3.0   0.0     0
8    2    3   66.666667    4.0   0.0     1
9    2    4   53.333333    5.0   0.0     0


In [169]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq_func_nb=call_seq_func1_nb)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq_func_nb=call_seq_func1_nb, row_wise=True)
print(portfolio.orders.records)

   col  idx        size  price  fees  side
0    1    0  100.000000    1.0   0.0     0
1    1    1  100.000000    2.0   0.0     1
2    1    2   66.666667    3.0   0.0     0
3    1    3   66.666667    4.0   0.0     1
4    1    4   53.333333    5.0   0.0     0
5    2    0  100.000000    1.0   0.0     0
6    2    1  100.000000    2.0   0.0     1
7    2    2   66.666667    3.0   0.0     0
8    2    3   66.666667    4.0   0.0     1
9    2    4   53.333333    5.0   0.0     0
   col  idx        size  price  fees  side
0    1    0  100.000000    1.0   0.0     0
1    1    1  100.000000    2.0   0.0     1
2    1    2   66.666667    3.0   0.0     0
3    1    3   66.666667    4.0   0.0     1
4    1    4   53.333333    5.0   0.0     0
5    2    0  100.000000    1.0   0.0     0
6    2    1  100.000000    2.0   0.0     1
7    2    2   66.666667    3.0   0.0     0
8    2    3   66.666667    4.0   0.0     1
9    2    4   53.333333    5.0   0.0     0


In [5]:
portfolio = vbt.Portfolio.from_order_func(
    price, order_inf_func_nb, price,
    target_shape=(5, 1)
)
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_inf_func_nb, price,
    target_shape=(5, 3)
)
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_inf_func_nb, price,
    target_shape=(5, 3), keys=pd.Index(['first', 'second', 'third'], name='custom')
)
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

Int64Index([0], dtype='int64')
2
Int64Index([0, 1, 2], dtype='int64', name='iteration_idx')
2
Index(['first', 'second', 'third'], dtype='object', name='custom')
2


In [11]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    active_mask=False)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    active_mask=[True, False, False])
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    active_mask=pd.Series([True, False, False, False, False]))
print(portfolio.orders.records)

Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    1    0  100.0    1.0   0.0     0
2    2    0  100.0    1.0   0.0     0


In [39]:
# active mask to 2d
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), active_mask=False)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), active_mask=[True, False])
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), active_mask=pd.Series([True, False, False, False, False]))
print(portfolio.orders.records)

Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
5    1    0  100.000000    1.0   0.0     0
6    1    1  100.000000    2.0   0.0     1
7    1    2   66.666667    3.0   0.0     0
8    1    3   66.666667    4.0   0.0     1
9    1    4   53.333333    5.0   0.0     0
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    1    0  100.0    1.0   0.0     0
2    2    0  100.0    1.0   0.0     0


In [40]:
# active mask to 1d
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), active_mask=False, row_wise=True)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_inf_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), active_mask=pd.Series([True, False, False, False, False]), row_wise=True)
print(portfolio.orders.records)

Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    1    0  100.0    1.0   0.0     0
2    2    0  100.0    1.0   0.0     0


In [36]:
@njit
def row_prep_func_nb(rc, seed):
    print(rc.i)
    np.random.seed(rc.i + seed)
    return (np.random.uniform(-10, 10, size=rc.target_shape[1]),)

@njit
def group_row_prep_func_nb(grc, seed):
    print(grc.i, grc.group)
    np.random.seed(grc.i + seed)
    return (np.random.uniform(-10, 10, size=grc.target_shape[1]),)

@njit
def order_func_nb(oc, target_values, price):
    if price.ndim == 1:  # flexible indexing
        order_price = price[oc.i]
    else:
        order_price = price[oc.i, oc.col]
    return vbt.portfolio.nb.Order(
        target_values[oc.col], 
        SizeType.Shares, 
        order_price, 
        0., 
        0., 
        0.
    )

In [42]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), row_prep_func_nb=group_row_prep_func_nb, row_prep_args=(42,))
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), row_prep_func_nb=row_prep_func_nb, row_prep_args=(42,), row_wise=True)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), row_prep_func_nb=group_row_prep_func_nb, row_prep_args=(42,),
    active_mask=pd.Series([True, False, False, False, False]))
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, price_wide, 
    group_by=np.array([0, 0, 1]), row_prep_func_nb=row_prep_func_nb, row_prep_args=(42,), row_wise=True, 
    active_mask=pd.Series([True, False, False, False, False]))
print(portfolio.orders.records)

0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
    col  idx      size  price  fees  side
0     0    2  6.696843    3.0   0.0     0
1     0    3  9.780230    4.0   0.0     0
2     0    4  5.676647    5.0   0.0     0
3     1    0  9.014286    1.0   0.0     0
4     1    1  2.181331    2.0   0.0     0
5     1    2  7.904078    3.0   0.0     1
6     1    3  0.990895    4.0   0.0     0
7     1    4  2.696674    5.0   0.0     0
8     2    0  4.639879    1.0   0.0     0
9     2    1  4.639879    2.0   0.0     1
10    2    2  4.892810    3.0   0.0     0
11    2    3  4.371054    4.0   0.0     1
12    2    4  0.521756    5.0   0.0     1
0
1
2
3
4
    col  idx      size  price  fees  side
0     0    2  6.696843    3.0   0.0     0
1     0    3  9.780230    4.0   0.0     0
2     0    4  5.676647    5.0   0.0     0
3     1    0  9.014286    1.0   0.0     0
4     1    1  2.181331    2.0   0.0     0
5     1    2  7.904078    3.0   0.0     1
6     1    3  0.990895    4.0   0.0     0
7     1    4  2.696674    

## methods and properties

In [20]:
vbt.Portfolio.active_returns.__qualname__

'Portfolio.active_returns'

In [21]:
vbt.Portfolio.sharpe_ratio.__qualname__

'add_returns_methods.<locals>.wrapper.<locals>.returns_method'

In [22]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1., 
    fees=0.01, fixed_fees=0.1, slippage=0.01,
    init_cash=[100., 100., 200.],
    accumulate=True, 
    accumulate_exit_mode=AccumulateExitMode.Reduce, 
    group_by=None
)  # independent
group_by = np.array(['first', 'first', 'second'])
portfolio_grouped = vbt.Portfolio.from_signals(
    price, entries, exits, size=1.,
    fees=0.01, fixed_fees=0.1, slippage=0.01,
    init_cash=[100., 100., 200.],
    accumulate=True, 
    accumulate_exit_mode=AccumulateExitMode.Reduce, 
    group_by=group_by,
    cash_sharing=False
)  # grouped
portfolio_shared = vbt.Portfolio.from_signals(
    price, entries, exits, size=1.,
    fees=0.01, fixed_fees=0.1, slippage=0.01,
    init_cash=[100., 200.],
    group_by=group_by,
    accumulate=True, 
    accumulate_exit_mode=AccumulateExitMode.Reduce, 
    cash_sharing=True
)  # shared

In [23]:
big_group_by = np.arange(1000)
big_portfolio = vbt.Portfolio.from_signals(
    big_price, big_entries, big_exits, size=1.)
big_portfolio_grouped = vbt.Portfolio.from_signals(
    big_price, big_entries, big_exits, size=1., group_by=big_group_by)
big_portfolio_shared = vbt.Portfolio.from_signals(
    big_price, big_entries, big_exits, size=1., group_by=big_group_by, cash_sharing=True)

In [39]:
print(portfolio.orders.records)
print(portfolio.orders.count())
print(portfolio.ref_price)
print(portfolio.init_cash)
print(portfolio.returns())
print(portfolio.total_return())

   col  idx  size  price    fees  side
0    0    1   1.0   2.02  0.1202     0
1    0    3   1.0   3.96  0.1396     1
2    1    0   1.0   1.01  0.1101     0
3    1    1   1.0   2.02  0.1202     0
4    1    3   1.0   3.96  0.1396     1
5    1    4   1.0   4.95  0.1495     1
6    2    0   1.0   1.01  0.1101     0
7    2    1   1.0   2.02  0.1202     0
8    2    3   1.0   3.96  0.1396     1
a    2
b    4
c    3
dtype: int64
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
a    100.0
b    100.0
c    200.0
dtype: float64
                   a         b         c
2020-01-01  0.000000 -0.120100 -0.120100
2020-01-02 -0.001402  0.008608  0.004302
2020-01-03  0.010014       NaN  0.009963
2020-01-04  0.008134       NaN  0.008979
2020-01-05  0.000000  0.007656       NaN
a    0.016802
b   -0.105731
c   -0.099497
dtype: float64


In [6]:
print(portfolio['a'].orders.records)
print(portfolio['a'].orders.count())
print(portfolio['a'].ref_price)
print(portfolio['a'].init_cash)
print(portfolio['a'].returns())
print(portfolio['a'].total_return())

   col  idx  size  price  fees  side
0    0    1   1.0    2.0  0.02     0
1    0    3   1.0    4.0  0.04     1
2
2020-01-01    NaN
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
Name: a, dtype: float64
100.0
2020-01-01    0.000000
2020-01-02   -0.000200
2020-01-03    0.010002
2020-01-04    0.009507
2020-01-05    0.000000
Name: a, dtype: float64
0.019399999999999862


In [7]:
print(portfolio['c'].orders.records)
print(portfolio['c'].orders.count())
print(portfolio['c'].ref_price)
print(portfolio['c'].init_cash)
print(portfolio['c'].returns())
print(portfolio['c'].total_return())

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    1   1.0    2.0  0.02     0
2    0    3   1.0    4.0  0.04     1
3
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
200.0
2020-01-01   -0.010000
2020-01-02    0.004900
2020-01-03    0.009952
2020-01-04    0.009657
2020-01-05         NaN
Name: c, dtype: float64
0.014454222711144604


In [8]:
print(portfolio[['c']].orders.records)
print(portfolio[['c']].orders.count())
print(portfolio[['c']].ref_price)
print(portfolio[['c']].init_cash)
print(portfolio[['c']].returns())
print(portfolio[['c']].total_return())

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    1   1.0    2.0  0.02     0
2    0    3   1.0    4.0  0.04     1
c    3
dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
c    200.0
dtype: float64
                   c
2020-01-01 -0.010000
2020-01-02  0.004900
2020-01-03  0.009952
2020-01-04  0.009657
2020-01-05       NaN
c    0.014454
dtype: float64


In [9]:
print(portfolio_grouped.orders.records)
print(portfolio_grouped.orders.count())
print(portfolio_grouped.ref_price)
print(portfolio_grouped.init_cash)
print(portfolio_grouped.returns())
print(portfolio_grouped.total_return())

   col  idx  size  price  fees  side
0    0    1   1.0    2.0  0.02     0
1    0    3   1.0    4.0  0.04     1
2    1    0   1.0    1.0  0.01     0
3    1    1   1.0    2.0  0.02     0
4    1    3   1.0    4.0  0.04     1
5    1    4   1.0    5.0  0.05     1
6    2    0   1.0    1.0  0.01     0
7    2    1   1.0    2.0  0.02     0
8    2    3   1.0    4.0  0.04     1
first     6
second    3
dtype: int64
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
a    100.0
b    100.0
c    200.0
dtype: float64
               first    second
2020-01-01 -0.010000 -0.010000
2020-01-02  0.004800  0.004900
2020-01-03       NaN  0.009952
2020-01-04       NaN  0.009657
2020-01-05  0.004592       NaN
first    -0.000680
second    0.014454
dtype: float64


In [10]:
print(portfolio_grouped['first'].orders.records)
print(portfolio_grouped['first'].orders.count())
print(portfolio_grouped['first'].ref_price)
print(portfolio_grouped['first'].init_cash)
print(portfolio_grouped['first'].returns())
print(portfolio_grouped['first'].total_return())

   col  idx  size  price  fees  side
0    0    1   1.0    2.0  0.02     0
1    0    3   1.0    4.0  0.04     1
2    1    0   1.0    1.0  0.01     0
3    1    1   1.0    2.0  0.02     0
4    1    3   1.0    4.0  0.04     1
5    1    4   1.0    5.0  0.05     1
6
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
a    100.0
b    100.0
dtype: float64
2020-01-01   -0.010000
2020-01-02    0.004800
2020-01-03         NaN
2020-01-04         NaN
2020-01-05    0.004592
Name: first, dtype: float64
-0.0006796054502687943


In [11]:
print(portfolio_grouped[['first']].orders.records)  # same because of collapse_group
print(portfolio_grouped[['first']].orders.count())
print(portfolio_grouped[['first']].ref_price)
print(portfolio_grouped[['first']].init_cash)
print(portfolio_grouped[['first']].returns())
print(portfolio_grouped[['first']].total_return())

   col  idx  size  price  fees  side
0    0    1   1.0    2.0  0.02     0
1    0    3   1.0    4.0  0.04     1
2    1    0   1.0    1.0  0.01     0
3    1    1   1.0    2.0  0.02     0
4    1    3   1.0    4.0  0.04     1
5    1    4   1.0    5.0  0.05     1
first    6
dtype: int64
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
a    100.0
b    100.0
dtype: float64
               first
2020-01-01 -0.010000
2020-01-02  0.004800
2020-01-03       NaN
2020-01-04       NaN
2020-01-05  0.004592
first   -0.00068
dtype: float64


In [12]:
print(portfolio_grouped['second'].orders.records)
print(portfolio_grouped['second'].orders.count())
print(portfolio_grouped['second'].ref_price)
print(portfolio_grouped['second'].init_cash)
print(portfolio_grouped['second'].returns())
print(portfolio_grouped['second'].total_return())

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    1   1.0    2.0  0.02     0
2    0    3   1.0    4.0  0.04     1
3
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
200.0
2020-01-01   -0.010000
2020-01-02    0.004900
2020-01-03    0.009952
2020-01-04    0.009657
2020-01-05         NaN
Name: second, dtype: float64
0.014454222711144604


In [13]:
print(portfolio_grouped[['second']].orders.records)
print(portfolio_grouped[['second']].orders.count())
print(portfolio_grouped[['second']].ref_price)
print(portfolio_grouped[['second']].init_cash)
print(portfolio_grouped[['second']].returns())
print(portfolio_grouped[['second']].total_return())

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    1   1.0    2.0  0.02     0
2    0    3   1.0    4.0  0.04     1
second    3
dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
c    200.0
dtype: float64
              second
2020-01-01 -0.010000
2020-01-02  0.004900
2020-01-03  0.009952
2020-01-04  0.009657
2020-01-05       NaN
second    0.014454
dtype: float64


In [14]:
print(portfolio_shared['first'].orders.records)
print(portfolio_shared['first'].orders.count())
print(portfolio_shared['first'].ref_price)
print(portfolio_shared['first'].init_cash)
print(portfolio_shared['first'].returns())
print(portfolio_shared['first'].total_return())

   col  idx  size  price  fees  side
0    0    1   1.0    2.0  0.02     0
1    0    3   1.0    4.0  0.04     1
2    1    0   1.0    1.0  0.01     0
3    1    1   1.0    2.0  0.02     0
4    1    3   1.0    4.0  0.04     1
5    1    4   1.0    5.0  0.05     1
6
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
100.0
2020-01-01   -0.010000
2020-01-02    0.009601
2020-01-03         NaN
2020-01-04         NaN
2020-01-05    0.008889
Name: first, dtype: float64
0.008389854611918102


In [15]:
print(portfolio_shared[['first']].orders.records)
print(portfolio_shared[['first']].orders.count())
print(portfolio_shared[['first']].ref_price)
print(portfolio_shared[['first']].init_cash)
print(portfolio_shared[['first']].returns())
print(portfolio_shared[['first']].total_return())

   col  idx  size  price  fees  side
0    0    1   1.0    2.0  0.02     0
1    0    3   1.0    4.0  0.04     1
2    1    0   1.0    1.0  0.01     0
3    1    1   1.0    2.0  0.02     0
4    1    3   1.0    4.0  0.04     1
5    1    4   1.0    5.0  0.05     1
first    6
dtype: int64
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
first    100.0
dtype: float64
               first
2020-01-01 -0.010000
2020-01-02  0.009601
2020-01-03       NaN
2020-01-04       NaN
2020-01-05  0.008889
first    0.00839
dtype: float64


In [16]:
print(portfolio_shared['second'].orders.records)
print(portfolio_shared['second'].orders.count())
print(portfolio_shared['second'].ref_price)
print(portfolio_shared['second'].init_cash)
print(portfolio_shared['second'].returns())
print(portfolio_shared['second'].total_return())

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    1   1.0    2.0  0.02     0
2    0    3   1.0    4.0  0.04     1
3
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
200.0
2020-01-01   -0.010000
2020-01-02    0.004900
2020-01-03    0.009952
2020-01-04    0.009657
2020-01-05         NaN
Name: second, dtype: float64
0.014454222711144604


In [17]:
print(portfolio_shared[['second']].orders.records)
print(portfolio_shared[['second']].orders.count())
print(portfolio_shared[['second']].ref_price)
print(portfolio_shared[['second']].init_cash)
print(portfolio_shared[['second']].returns())
print(portfolio_shared[['second']].total_return())

   col  idx  size  price  fees  side
0    0    0   1.0    1.0  0.01     0
1    0    1   1.0    2.0  0.02     0
2    0    3   1.0    4.0  0.04     1
second    3
dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
second    200.0
dtype: float64
              second
2020-01-01 -0.010000
2020-01-02  0.004900
2020-01-03  0.009952
2020-01-04  0.009657
2020-01-05       NaN
second    0.014454
dtype: float64


In [16]:
%timeit big_portfolio.iloc[0]
%timeit big_portfolio.iloc[:]
%timeit big_portfolio_grouped.iloc[0]
%timeit big_portfolio_grouped.iloc[:]
%timeit big_portfolio_shared.iloc[0]
%timeit big_portfolio_shared.iloc[:]

4.2 ms ± 75 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
55.6 ms ± 26.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
5.24 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
41.8 ms ± 958 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
5.1 ms ± 99.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
41.8 ms ± 383 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
print(portfolio.init_cash)
print(portfolio_grouped.init_cash)
print(portfolio_shared.init_cash)

a    100.0
b    100.0
c    200.0
dtype: float64
a    100.0
b    100.0
c    200.0
dtype: float64
first     100.0
second    200.0
dtype: float64


In [6]:
print(portfolio.regroup_init_cash())
print(portfolio_grouped.regroup_init_cash(group_by=False))
print(portfolio_shared.regroup_init_cash(group_by=False))

print(portfolio.regroup_init_cash(group_by=group_by))
print(portfolio_grouped.regroup_init_cash())
print(portfolio_shared.regroup_init_cash())

a    100.0
b    100.0
c    200.0
dtype: float64
a    100.0
b    100.0
c    200.0
dtype: float64
a    100.0
b    100.0
c    200.0
dtype: float64
first     200.0
second    200.0
dtype: float64
first     200.0
second    200.0
dtype: float64
first     100.0
second    200.0
dtype: float64


In [6]:
%timeit big_portfolio.regroup_init_cash()
%timeit big_portfolio_grouped.regroup_init_cash(group_by=False)
%timeit big_portfolio_shared.regroup_init_cash(group_by=False)

%timeit big_portfolio.regroup_init_cash(group_by=big_group_by)
%timeit big_portfolio_grouped.regroup_init_cash()
%timeit big_portfolio_shared.regroup_init_cash()

83.1 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
86.7 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.08 ms ± 21.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.48 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.36 ms ± 29.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
965 µs ± 9.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
print(portfolio.cash)
print(portfolio_grouped.cash)
print(portfolio_shared.cash)

                 a       b       c
2020-01-01  100.00   98.99  198.99
2020-01-02   97.98   96.97  196.97
2020-01-03   97.98   96.97  196.97
2020-01-04  101.94  100.93  200.93
2020-01-05  101.94  105.88  200.93
                 a       b       c
2020-01-01  100.00   98.99  198.99
2020-01-02   97.98   96.97  196.97
2020-01-03   97.98   96.97  196.97
2020-01-04  101.94  100.93  200.93
2020-01-05  101.94  105.88  200.93
                 a       b       c
2020-01-01  100.00   98.99  198.99
2020-01-02   96.97   94.95  196.97
2020-01-03   94.95   94.95  196.97
2020-01-04   98.91  102.87  200.93
2020-01-05  102.87  107.82  200.93


In [5]:
print(portfolio.regroup_cash())
print(portfolio_grouped.regroup_cash(group_by=False))
print(portfolio_shared.regroup_cash(group_by=False))

print(portfolio.regroup_cash(group_by=group_by))
print(portfolio_grouped.regroup_cash())
print(portfolio_shared.regroup_cash())

                   a         b         c
2020-01-01  100.0000   98.8799  198.8799
2020-01-02   97.8598   96.7397  196.7397
2020-01-03   97.8598   96.7397  196.7397
2020-01-04  101.6802  100.5601  200.5601
2020-01-05  101.6802  105.3606  200.5601
                   a         b         c
2020-01-01  100.0000   98.8799  198.8799
2020-01-02   97.8598   96.7397  196.7397
2020-01-03   97.8598   96.7397  196.7397
2020-01-04  101.6802  100.5601  200.5601
2020-01-05  101.6802  105.3606  200.5601
                   a         b         c
2020-01-01  100.0000   98.8799  198.8799
2020-01-02   96.7397   94.5995  196.7397
2020-01-03   94.5995   94.5995  196.7397
2020-01-04   98.4199  102.2403  200.5601
2020-01-05  102.2403  107.0408  200.5601
               first    second
2020-01-01  198.8799  198.8799
2020-01-02  194.5995  196.7397
2020-01-03  194.5995  196.7397
2020-01-04  202.2403  200.5601
2020-01-05  207.0408  200.5601
               first    second
2020-01-01  198.8799  198.8799
2020-01-02  19

In [8]:
%timeit big_portfolio.regroup_cash()
%timeit big_portfolio_grouped.regroup_cash(group_by=False)
%timeit big_portfolio_shared.regroup_cash(group_by=False)

%timeit big_portfolio.regroup_cash(group_by=big_group_by)
%timeit big_portfolio_grouped.regroup_cash()
%timeit big_portfolio_shared.regroup_cash()

109 µs ± 8.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
107 µs ± 911 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
110 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
9.38 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.15 ms ± 63.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.88 ms ± 47.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
print(portfolio.shares)
print(portfolio_grouped.shares)
print(portfolio_shared.shares)

              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  1.0  2.0  2.0
2020-01-03  1.0  2.0  2.0
2020-01-04  0.0  1.0  1.0
2020-01-05  0.0  0.0  1.0
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  1.0  2.0  2.0
2020-01-03  1.0  2.0  2.0
2020-01-04  0.0  1.0  1.0
2020-01-05  0.0  0.0  1.0
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  1.0  2.0  2.0
2020-01-03  1.0  2.0  2.0
2020-01-04  0.0  1.0  1.0
2020-01-05  0.0  0.0  1.0


In [14]:
print(portfolio.ref_price)
print(portfolio_grouped.ref_price)
print(portfolio_shared.ref_price)

              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN


In [15]:
print(portfolio.fill_ref_price(ffill=False, bfill=False))
print(portfolio.fill_ref_price(ffill=True, bfill=False))
print(portfolio.fill_ref_price(ffill=False, bfill=True))
print(portfolio.fill_ref_price(ffill=True, bfill=True))

              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  2.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  4.0
              a    b    c
2020-01-01  2.0  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  4.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  2.0  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  2.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  4.0


In [10]:
%timeit big_portfolio.get_ref_price(ffill=True, bfill=True)

125 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
print(portfolio.orders.count())
print(portfolio_grouped.orders.count())
print(portfolio_shared.orders.count())

a    2
b    4
c    3
dtype: int64
first     6
second    3
dtype: int64
first     6
second    3
dtype: int64


In [17]:
print(portfolio.regroup_orders().count())
print(portfolio_grouped.regroup_orders(group_by=False).count())
print(portfolio_shared.regroup_orders(group_by=False).count())

print(portfolio.regroup_orders(group_by=group_by).count())
print(portfolio_grouped.regroup_orders().count())
print(portfolio_shared.regroup_orders().count())

a    2
b    4
c    3
dtype: int64
a    2
b    4
c    3
dtype: int64
a    2
b    4
c    3
dtype: int64
first     6
second    3
dtype: int64
first     6
second    3
dtype: int64
first     6
second    3
dtype: int64


In [18]:
print(portfolio.trades(incl_unrealized=True).count())

print(portfolio.trades().count())
print(portfolio_grouped.trades(group_by=False).count())
print(portfolio_shared.trades(group_by=False).count())

print(portfolio.trades(group_by=group_by).count())
print(portfolio_grouped.trades().count())
print(portfolio_shared.trades().count())

a    1
b    2
c    2
dtype: int64
a    1
b    2
c    1
dtype: int64
a    1
b    2
c    1
dtype: int64
a    1
b    2
c    1
dtype: int64
first     3
second    1
dtype: int64
first     3
second    1
dtype: int64
first     3
second    1
dtype: int64


In [41]:
%timeit big_portfolio.trades()

66.4 ms ± 27.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
print(portfolio.positions(incl_unrealized=True).count())

print(portfolio.positions().count())
print(portfolio_grouped.positions(group_by=False).count())
print(portfolio_shared.positions(group_by=False).count())

print(portfolio.positions(group_by=group_by).count())
print(portfolio_grouped.positions().count())
print(portfolio_shared.positions().count())

a    1
b    1
c    1
dtype: int64
a    1
b    1
c    0
dtype: int64
a    1
b    1
c    0
dtype: int64
a    1
b    1
c    0
dtype: int64
first     2
second    0
dtype: int64
first     2
second    0
dtype: int64
first     2
second    0
dtype: int64


In [39]:
%timeit big_portfolio.positions()

42.3 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
print(portfolio.drawdowns().count())
print(portfolio_grouped.drawdowns(group_by=False).count())
print(portfolio_shared.drawdowns(group_by=False).count())

print(portfolio.drawdowns(group_by=group_by).count())
print(portfolio_grouped.drawdowns().count())
print(portfolio_shared.drawdowns().count())

a    1
b    0
c    0
dtype: int64
a    1
b    0
c    0
dtype: int64
a    1
b    0
c    0
dtype: int64
first     0
second    0
dtype: int64
first     0
second    0
dtype: int64
first     0
second    0
dtype: int64


In [43]:
%timeit big_portfolio.drawdowns()

16.6 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
print(portfolio.cash_flow())
print(portfolio_grouped.cash_flow(group_by=False))
print(portfolio_shared.cash_flow(group_by=False))

print(portfolio.cash_flow(group_by=group_by))
print(portfolio_grouped.cash_flow())
print(portfolio_shared.cash_flow())

                 a       b       c
2020-01-01  0.0000 -1.1201 -1.1201
2020-01-02 -2.1402 -2.1402 -2.1402
2020-01-03  0.0000  0.0000  0.0000
2020-01-04  3.8204  3.8204  3.8204
2020-01-05  0.0000  4.8005  0.0000
                 a       b       c
2020-01-01  0.0000 -1.1201 -1.1201
2020-01-02 -2.1402 -2.1402 -2.1402
2020-01-03  0.0000  0.0000  0.0000
2020-01-04  3.8204  3.8204  3.8204
2020-01-05  0.0000  4.8005  0.0000
                 a       b       c
2020-01-01  0.0000 -1.1201 -1.1201
2020-01-02 -2.1402 -2.1402 -2.1402
2020-01-03  0.0000  0.0000  0.0000
2020-01-04  3.8204  3.8204  3.8204
2020-01-05  0.0000  4.8005  0.0000
             first  second
2020-01-01 -1.1201 -1.1201
2020-01-02 -4.2804 -2.1402
2020-01-03  0.0000  0.0000
2020-01-04  7.6408  3.8204
2020-01-05  4.8005  0.0000
             first  second
2020-01-01 -1.1201 -1.1201
2020-01-02 -4.2804 -2.1402
2020-01-03  0.0000  0.0000
2020-01-04  7.6408  3.8204
2020-01-05  4.8005  0.0000
             first  second
2020-01-01 -1.1201 

In [8]:
%timeit big_portfolio.cash_flow()
%timeit big_portfolio_grouped.cash_flow(group_by=False)
%timeit big_portfolio_shared.cash_flow(group_by=False)

%timeit big_portfolio.cash_flow(group_by=big_group_by)
%timeit big_portfolio_grouped.cash_flow()
%timeit big_portfolio_shared.cash_flow()

8.26 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.36 ms ± 93.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.69 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.4 ms ± 85.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.5 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
18 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
print(portfolio.holding_value())
print(portfolio_grouped.holding_value(group_by=False))
print(portfolio_shared.holding_value(group_by=False))

print(portfolio.holding_value(group_by=group_by))
print(portfolio_grouped.holding_value())
print(portfolio_shared.holding_value())

              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  2.0  4.0  4.0
2020-01-03  3.0  NaN  6.0
2020-01-04  0.0  4.0  4.0
2020-01-05  0.0  0.0  NaN
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  2.0  4.0  4.0
2020-01-03  3.0  NaN  6.0
2020-01-04  0.0  4.0  4.0
2020-01-05  0.0  0.0  NaN
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  2.0  4.0  4.0
2020-01-03  3.0  NaN  6.0
2020-01-04  0.0  4.0  4.0
2020-01-05  0.0  0.0  NaN
            first  second
2020-01-01    1.0     1.0
2020-01-02    6.0     4.0
2020-01-03    NaN     6.0
2020-01-04    4.0     4.0
2020-01-05    0.0     NaN
            first  second
2020-01-01    1.0     1.0
2020-01-02    6.0     4.0
2020-01-03    NaN     6.0
2020-01-04    4.0     4.0
2020-01-05    0.0     NaN
            first  second
2020-01-01    1.0     1.0
2020-01-02    6.0     4.0
2020-01-03    NaN     6.0
2020-01-04    4.0     4.0
2020-01-05    0.0     NaN


In [11]:
%timeit big_portfolio.holding_value()
%timeit big_portfolio_grouped.holding_value(group_by=False)
%timeit big_portfolio_shared.holding_value(group_by=False)

%timeit big_portfolio.holding_value(group_by=big_group_by)
%timeit big_portfolio_grouped.holding_value()
%timeit big_portfolio_shared.holding_value()

7.25 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.52 ms ± 56.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.54 ms ± 49.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
22.8 ms ± 99.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
22.9 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
24.4 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
print(portfolio.value())
print(portfolio_grouped.value(group_by=False))
print(portfolio_shared.value(group_by=False))

print(portfolio_shared.value(group_by=False, iterative=True))

print(portfolio.value(group_by=group_by))
print(portfolio_grouped.value())
print(portfolio_shared.value())

                 a       b       c
2020-01-01  100.00   99.99  199.99
2020-01-02   99.98  100.97  200.97
2020-01-03  100.98     NaN  202.97
2020-01-04  101.94  104.93  204.93
2020-01-05  101.94  105.88     NaN
                 a       b       c
2020-01-01  100.00   99.99  199.99
2020-01-02   99.98  100.97  200.97
2020-01-03  100.98     NaN  202.97
2020-01-04  101.94  104.93  204.93
2020-01-05  101.94  105.88     NaN
                 a       b       c
2020-01-01  100.00   99.99  199.99
2020-01-02   99.98  100.97  200.97
2020-01-03  100.98     NaN  202.97
2020-01-04  101.94  104.93  204.93
2020-01-05  101.94  105.88     NaN
                 a       b       c
2020-01-01  100.00   99.99  199.99
2020-01-02   99.97  100.95  200.97
2020-01-03  101.95     NaN  202.97
2020-01-04     NaN  106.87  204.93
2020-01-05  106.87  107.82     NaN
             first  second
2020-01-01  199.99  199.99
2020-01-02  200.95  200.97
2020-01-03     NaN  202.97
2020-01-04  206.87  204.93
2020-01-05  207.82     Na

In [6]:
%timeit big_portfolio.value()
%timeit big_portfolio_grouped.value(group_by=False)
%timeit big_portfolio_shared.value(group_by=False)

%timeit big_portfolio_shared.value(group_by=False, iterative=True)

%timeit big_portfolio.value(group_by=big_group_by)
%timeit big_portfolio_grouped.value()
%timeit big_portfolio_shared.value()

9.39 ms ± 69 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
38.9 ms ± 506 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
44.3 ms ± 584 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
28.1 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.7 ms ± 232 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.9 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.5 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
print(portfolio.final_value())
print(portfolio_grouped.final_value(group_by=False))
print(portfolio_shared.final_value(group_by=False))

print(portfolio_shared.final_value(group_by=False, iterative=True))

print(portfolio.final_value(group_by=group_by))
print(portfolio_grouped.final_value())
print(portfolio_shared.final_value())

a    101.94
b    105.88
c    204.93
dtype: float64
a    101.94
b    105.88
c    204.93
dtype: float64
a    101.94
b    105.88
c    204.93
dtype: float64
a    106.87
b    107.82
c    204.93
dtype: float64
first     207.82
second    204.93
dtype: float64
first     207.82
second    204.93
dtype: float64
first     107.82
second    204.93
dtype: float64


In [8]:
%timeit big_portfolio.final_value()
%timeit big_portfolio_grouped.final_value(group_by=False)
%timeit big_portfolio_shared.final_value(group_by=False)

%timeit big_portfolio_shared.final_value(group_by=False, iterative=True)

%timeit big_portfolio.final_value(group_by=big_group_by)
%timeit big_portfolio_grouped.final_value()
%timeit big_portfolio_shared.final_value()

11.8 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
41.3 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
45.7 ms ± 438 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.1 ms ± 6.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
39.9 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
39.3 ms ± 4.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
37 ms ± 316 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
print(portfolio.total_profit())
print(portfolio_grouped.total_profit(group_by=False))
print(portfolio_shared.total_profit(group_by=False))

print(portfolio.total_profit(group_by=group_by))
print(portfolio_grouped.total_profit())
print(portfolio_shared.total_profit())

a    1.6802
b    5.3606
c    4.5601
dtype: float64
a    1.6802
b    5.3606
c    4.5601
dtype: float64
a    1.6802
b    5.3606
c    4.5601
dtype: float64
first     7.0408
second    4.5601
dtype: float64
first     7.0408
second    4.5601
dtype: float64
first     7.0408
second    4.5601
dtype: float64


In [19]:
%timeit big_portfolio.total_profit()
%timeit big_portfolio_grouped.total_profit()
%timeit big_portfolio_grouped.total_profit(group_by=False)

18.1 ms ± 571 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
45.4 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
57.4 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
print(portfolio.returns())
print(portfolio_grouped.returns(group_by=False))
print(portfolio_shared.returns(group_by=False))

print(portfolio_shared.returns(group_by=False, iterative=True))

print(portfolio.returns(group_by=group_by))
print(portfolio_grouped.returns())
print(portfolio_shared.returns())  # twice bigger returns because of twice smaller init cash

                   a         b         c
2020-01-01  0.000000 -0.010000 -0.010000
2020-01-02 -0.000200  0.009801  0.004900
2020-01-03  0.010002       NaN  0.009952
2020-01-04  0.009507       NaN  0.009657
2020-01-05  0.000000  0.009054       NaN
                   a         b         c
2020-01-01  0.000000 -0.010000 -0.010000
2020-01-02 -0.000200  0.009801  0.004900
2020-01-03  0.010002       NaN  0.009952
2020-01-04  0.009507       NaN  0.009657
2020-01-05  0.000000  0.009054       NaN
                   a         b         c
2020-01-01  0.000000 -0.010000 -0.010000
2020-01-02 -0.000200  0.009801  0.004900
2020-01-03  0.010002       NaN  0.009952
2020-01-04  0.009507       NaN  0.009657
2020-01-05  0.000000  0.009054       NaN
                   a         b         c
2020-01-01  0.000000 -0.000100  0.999900
2020-01-02 -0.000200  0.009803  0.004900
2020-01-03  0.009906       NaN  0.009952
2020-01-04       NaN       NaN  0.009657
2020-01-05  0.000000  0.008889       NaN
               f

In [8]:
%timeit big_portfolio.returns()
%timeit big_portfolio_grouped.returns(group_by=False)
%timeit big_portfolio_shared.returns(group_by=False)

%timeit big_portfolio_shared.returns(group_by=False, iterative=True)

%timeit big_portfolio.returns(group_by=big_group_by)
%timeit big_portfolio_grouped.returns()
%timeit big_portfolio_shared.returns()

17.7 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
39.6 ms ± 4.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
59.7 ms ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
49.9 ms ± 379 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
48.9 ms ± 3.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
53.7 ms ± 6.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
48.8 ms ± 3.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
print(portfolio.active_returns())
print(portfolio_grouped.active_returns(group_by=False))
print(portfolio_shared.active_returns(group_by=False))

print(portfolio.active_returns(group_by=group_by))
print(portfolio_grouped.active_returns())
print(portfolio_shared.active_returns())

                   a         b         c
2020-01-01  0.000000 -0.009901 -0.009901
2020-01-02 -0.009901  0.324503  0.324503
2020-01-03  0.500000       NaN  0.500000
2020-01-04  0.320000       NaN  0.326667
2020-01-05  0.000000  0.237500       NaN
                   a         b         c
2020-01-01  0.000000 -0.009901 -0.009901
2020-01-02 -0.009901  0.324503  0.324503
2020-01-03  0.500000       NaN  0.500000
2020-01-04  0.320000       NaN  0.326667
2020-01-05  0.000000  0.237500       NaN
                   a         b         c
2020-01-01  0.000000 -0.009901 -0.009901
2020-01-02 -0.009901  0.324503  0.324503
2020-01-03  0.500000       NaN  0.500000
2020-01-04  0.320000       NaN  0.326667
2020-01-05  0.000000  0.237500       NaN
               first    second
2020-01-01 -0.009901 -0.009901
2020-01-02  0.190476  0.324503
2020-01-03       NaN  0.500000
2020-01-04       NaN  0.326667
2020-01-05  0.237500       NaN
               first    second
2020-01-01 -0.009901 -0.009901
2020-01-02  0.

In [28]:
%timeit big_portfolio.active_returns()
%timeit big_portfolio_grouped.active_returns(group_by=False)
%timeit big_portfolio_shared.active_returns(group_by=False)

%timeit big_portfolio.active_returns(group_by=big_group_by)
%timeit big_portfolio_grouped.active_returns()
%timeit big_portfolio_shared.active_returns()

34.1 ms ± 279 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.8 ms ± 844 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
51.5 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
64.5 ms ± 4.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
62.4 ms ± 394 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
66.2 ms ± 7.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
print(portfolio.buy_and_hold_return())
print(portfolio_grouped.buy_and_hold_return(group_by=False))
print(portfolio_shared.buy_and_hold_return(group_by=False))

print(portfolio.buy_and_hold_return(group_by=group_by))
print(portfolio_grouped.buy_and_hold_return())
print(portfolio_shared.buy_and_hold_return())

a    1.5
b    4.0
c    3.0
dtype: float64
a    1.5
b    4.0
c    3.0
dtype: float64
a    1.5
b    4.0
c    3.0
dtype: float64
first     2.75
second    3.00
dtype: float64
first     2.75
second    3.00
dtype: float64
first     2.75
second    3.00
dtype: float64


In [6]:
%timeit big_portfolio.buy_and_hold_return()
%timeit big_portfolio_grouped.buy_and_hold_return(group_by=False)
%timeit big_portfolio_shared.buy_and_hold_return(group_by=False)

%timeit big_portfolio.buy_and_hold_return(group_by=big_group_by)
%timeit big_portfolio_grouped.buy_and_hold_return()
%timeit big_portfolio_shared.buy_and_hold_return()

290 µs ± 56.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
243 µs ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
237 µs ± 8.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
737 µs ± 282 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
567 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.83 ms ± 92.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
print(portfolio_shared.cumulative_returns())
print(portfolio_shared.cumulative_returns(group_by=False))
print(portfolio_shared.cumulative_returns(group_by=False, returns_kwargs={'iterative': True}))
print(portfolio_shared.cumulative_returns(group_by=False, returns_kwargs={'active': True}))

               first    second
2020-01-01 -0.010000 -0.010000
2020-01-02 -0.000495 -0.005149
2020-01-03 -0.000495  0.004752
2020-01-04 -0.000495  0.014454
2020-01-05  0.008390  0.014454
                 a         b         c
2020-01-01  0.0000 -0.010000 -0.010000
2020-01-02 -0.0002 -0.000297 -0.005149
2020-01-03  0.0098 -0.000297  0.004752
2020-01-04  0.0194 -0.000297  0.014454
2020-01-05  0.0194  0.008754  0.014454
                   a         b       c
2020-01-01  0.000000 -0.000100  0.9999
2020-01-02 -0.000200  0.009702  1.0097
2020-01-03  0.009704  0.009702  1.0297
2020-01-04  0.009704  0.009702  1.0493
2020-01-05  0.009704  0.018678  1.0493
                   a         b         c
2020-01-01  0.000000 -0.009901 -0.009901
2020-01-02 -0.009901  0.311389  0.311389
2020-01-03  0.485149  0.311389  0.967084
2020-01-04  0.960396  0.311389  1.609665
2020-01-05  0.960396  0.622844  1.609665


In [31]:
print(portfolio_shared.sharpe_ratio())
print(portfolio_shared.sharpe_ratio(year_freq='252D'))
print(portfolio_shared.sharpe_ratio(group_by=False))
print(portfolio_shared.sharpe_ratio(group_by=False, returns_kwargs={'iterative': True}))
print(portfolio_shared.sharpe_ratio(group_by=False, returns_kwargs={'active': True}))

first     4.863664
second    7.391588
dtype: float64
first     4.041266
second    6.141743
dtype: float64
a    13.706690
b     5.024615
c     7.391588
dtype: float64
a     9.295366
b    21.634449
c     9.867139
dtype: float64
a    13.161545
b    20.265523
c    25.555721
dtype: float64


In [32]:
print(portfolio['a'].stats())

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                       40
Total Profit                             1.94
Total Return [%]                         1.94
Buy & Hold Return [%]                     150
Max. Drawdown [%]                        0.02
Avg. Drawdown [%]                        0.02
Max. Drawdown Duration        2 days 00:00:00
Avg. Drawdown Duration        2 days 00:00:00
Num. Trades                                 1
Win Rate [%]                              100
Best Trade [%]                        96.0396
Worst Trade [%]                       96.0396
Avg. Trade [%]                        96.0396
Max. Trade Duration           2 days 00:00:00
Avg. Trade Duration           2 days 00:00:00
Expectancy                               1.94
SQN                                       NaN
Sharpe Ratio                          13.7067
Sortino Ratio                     

In [33]:
print(portfolio['a'].stats(incl_unrealized=True))

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                       40
Total Profit                             1.94
Total Return [%]                         1.94
Buy & Hold Return [%]                     150
Max. Drawdown [%]                        0.02
Avg. Drawdown [%]                        0.02
Max. Drawdown Duration        2 days 00:00:00
Avg. Drawdown Duration        2 days 00:00:00
Num. Trades                                 1
Win Rate [%]                              100
Best Trade [%]                        96.0396
Worst Trade [%]                       96.0396
Avg. Trade [%]                        96.0396
Max. Trade Duration           2 days 00:00:00
Avg. Trade Duration           2 days 00:00:00
Expectancy                               1.94
SQN                                       NaN
Sharpe Ratio                          13.7067
Sortino Ratio                     

In [11]:
print(portfolio.stats(column='c'))

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                        0
Total Profit                             4.93
Total Return [%]                      1.44542
Buy & Hold Return [%]                     300
Max. Drawdown [%]                          -0
Avg. Drawdown [%]                          -0
Max. Drawdown Duration                    NaT
Avg. Drawdown Duration                    NaT
Num. Trades                                 1
Win Rate [%]                              100
Best Trade [%]                        161.386
Worst Trade [%]                       161.386
Avg. Trade [%]                        161.386
Max. Trade Duration           3 days 00:00:00
Avg. Trade Duration           3 days 00:00:00
Expectancy                              2.445
SQN                                       NaN
Sharpe Ratio                          7.39159
Sortino Ratio                     

In [5]:
print(portfolio_grouped['second'].stats())

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                        0
Total Profit                             4.93
Total Return [%]                      1.44542
Buy & Hold Return [%]                     300
Max. Drawdown [%]                          -0
Avg. Drawdown [%]                          -0
Max. Drawdown Duration                    NaT
Avg. Drawdown Duration                    NaT
Num. Trades                                 1
Win Rate [%]                              100
Best Trade [%]                        161.386
Worst Trade [%]                       161.386
Avg. Trade [%]                        161.386
Max. Trade Duration           3 days 00:00:00
Avg. Trade Duration           3 days 00:00:00
Expectancy                              2.445
SQN                                       NaN
Sharpe Ratio                          7.39159
Sortino Ratio                     

In [6]:
print(portfolio_grouped.stats(column='second'))

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                        0
Total Profit                             4.93
Total Return [%]                      1.44542
Buy & Hold Return [%]                     300
Max. Drawdown [%]                          -0
Avg. Drawdown [%]                          -0
Max. Drawdown Duration                    NaT
Avg. Drawdown Duration                    NaT
Num. Trades                                 1
Win Rate [%]                              100
Best Trade [%]                        161.386
Worst Trade [%]                       161.386
Avg. Trade [%]                        161.386
Max. Trade Duration           3 days 00:00:00
Avg. Trade Duration           3 days 00:00:00
Expectancy                              2.445
SQN                                       NaN
Sharpe Ratio                          7.39159
Sortino Ratio                     

In [7]:
print(portfolio_grouped.stats(column='c', group_by=False))

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                        0
Total Profit                             4.93
Total Return [%]                      1.44542
Buy & Hold Return [%]                     300
Max. Drawdown [%]                          -0
Avg. Drawdown [%]                          -0
Max. Drawdown Duration                    NaT
Avg. Drawdown Duration                    NaT
Num. Trades                                 1
Win Rate [%]                              100
Best Trade [%]                        161.386
Worst Trade [%]                       161.386
Avg. Trade [%]                        161.386
Max. Trade Duration           3 days 00:00:00
Avg. Trade Duration           3 days 00:00:00
Expectancy                              2.445
SQN                                       NaN
Sharpe Ratio                          7.39159
Sortino Ratio                     

In [17]:
# Calculates stats for one column
%timeit big_portfolio.iloc[:, 0].stats()
%timeit big_portfolio_grouped.iloc[:, 0].stats(group_by=False)
%timeit big_portfolio_shared.iloc[:, 0].stats(group_by=False)

%timeit big_portfolio.iloc[:, 0].stats(group_by=np.array([0]))
%timeit big_portfolio_grouped.iloc[:, 0].stats()
%timeit big_portfolio_shared.iloc[:, 0].stats()

19.9 ms ± 775 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
23.4 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
41.6 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
28.4 ms ± 446 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
29.4 ms ± 862 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
53.1 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
# Calculates stats for all columns and selects one, takes advantage of caching (which is disabled here)
# Thus some series are re-calculated multiple times, such as records and returns
%timeit big_portfolio.stats(column=0)
%timeit big_portfolio_grouped.stats(group_by=False, column=0)
%timeit big_portfolio_shared.stats(group_by=False, column=0)

%timeit big_portfolio.stats(group_by=big_group_by, column=0)
%timeit big_portfolio_grouped.stats(column=0)
%timeit big_portfolio_shared.stats(column=0)

451 ms ± 72.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
534 ms ± 27.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
655 ms ± 26.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
591 ms ± 31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
608 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
600 ms ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
